Cell 1: Install Libraries

In [ ]:
# Install required packages for Hugging Face, PyTorch, and metrics
!pip install transformers datasets torch pandas scikit-learn

# Import libraries
import pandas as pd
import numpy as np
from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import f1_score, accuracy_score
from sklearn.utils import resample
from google.colab import files
import re
import torch

print("Libraries installed and imported.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Cell 2: Load and Combine Datasets

In [ ]:
davidson = pd.read_csv("labeled_data.csv")[["tweet", "class"]]
davidson_dataset = Dataset.from_pandas(davidson)
print("Davidson dataset loaded:", len(davidson_dataset), "rows")
print("Davidson sample:", davidson_dataset[0])

# Load HateXplain
hatexplain = load_dataset("hatexplain")["train"]
hatexplain = hatexplain.map(lambda x: {
    "tweet": " ".join(x["post_tokens"]),
    "class": x["annotators"]["label"][0]
})
hatexplain = hatexplain.select_columns(["tweet", "class"])
print("HateXplain dataset loaded:", len(hatexplain), "rows")
print("HateXplain sample:", hatexplain[0])

# Combine datasets
combined_dataset = concatenate_datasets([davidson_dataset, hatexplain])
print("Combined dataset:", len(combined_dataset), "rows")
print("Combined sample:", combined_dataset[0])

Davidson dataset loaded: 24783 rows
Davidson sample: {'tweet': "!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...", 'class': 2}


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

hatexplain.py:   0%|          | 0.00/4.78k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15383 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1922 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1924 [00:00<?, ? examples/s]

Map:   0%|          | 0/15383 [00:00<?, ? examples/s]

HateXplain dataset loaded: 15383 rows
HateXplain sample: {'tweet': 'u really think i would not have been raped by feral hindu or muslim back in india or bangladesh and a neo nazi would rape me as well just to see me cry', 'class': 0}
Combined dataset: 40166 rows
Combined sample: {'tweet': "!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...", 'class': 2}


Cell 3: Balance Classes

In [ ]:
# Convert to pandas
df = combined_dataset.to_pandas()
print("Original class distribution:")
print(df["class"].value_counts())

# Balance to ~14k per class
class_0 = resample(df[df["class"] == 0], n_samples=14000, replace=True, random_state=42)
class_1 = resample(df[df["class"] == 1], n_samples=14000, random_state=42)
class_2 = resample(df[df["class"] == 2], n_samples=14000, replace=True, random_state=42)
balanced_df = pd.concat([class_0, class_1, class_2])

print("Balanced class distribution:")
print(balanced_df["class"].value_counts())

# Convert back
balanced_dataset = Dataset.from_pandas(balanced_df)
print("Balanced dataset:", len(balanced_dataset), "rows")
print("Balanced sample:", balanced_dataset[0])

Original class distribution:
class
1    25489
2     8677
0     6000
Name: count, dtype: int64
Balanced class distribution:
class
0    14000
1    14000
2    14000
Name: count, dtype: int64
Balanced dataset: 42000 rows
Balanced sample: {'tweet': "Jennifer Lawrence's nudes sooooo trash. Im still smashin though, no doubt", 'class': 0, '__index_level_0__': 12052}


Cell 4: Split Dataset

In [ ]:
# Split: 80% train, 10% val, 10% test
dataset = balanced_dataset.train_test_split(test_size=0.2, seed=42)
test_val = dataset["test"].train_test_split(test_size=0.5, seed=42)
train_dataset = dataset["train"]
val_dataset = test_val["train"]
test_dataset = test_val["test"]

print("Train dataset:", len(train_dataset), "rows")
print("Validation dataset:", len(val_dataset), "rows")
print("Test dataset:", len(test_dataset), "rows")
print("Train sample:", train_dataset[0])

Train dataset: 33600 rows
Validation dataset: 4200 rows
Test dataset: 4200 rows
Train sample: {'tweet': 'bill the redneck bought a diesel truck and then got in a fight before waking up to creepy ghost sounds', 'class': 1, '__index_level_0__': 36731}


Cell 5: Preprocess Text

In [ ]:
# Clean text
def clean_text(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    return text.strip()

train_dataset = train_dataset.map(lambda x: {"tweet": clean_text(x["tweet"])})
val_dataset = val_dataset.map(lambda x: {"tweet": clean_text(x["tweet"])})
test_dataset = test_dataset.map(lambda x: {"tweet": clean_text(x["tweet"])})

print("Text cleaning completed.")
print("Train sample after cleaning:", train_dataset[0])

Map:   0%|          | 0/33600 [00:00<?, ? examples/s]

Map:   0%|          | 0/4200 [00:00<?, ? examples/s]

Map:   0%|          | 0/4200 [00:00<?, ? examples/s]

Text cleaning completed.
Train sample after cleaning: {'tweet': 'bill the redneck bought a diesel truck and then got in a fight before waking up to creepy ghost sounds', 'class': 1, '__index_level_0__': 36731}


Cell 6: Tokenize Data

In [ ]:
# Load tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize, preserve labels
def tokenize_function(examples):
    # Tokenize and include class as labels
    tokenized = tokenizer(examples["tweet"], padding="max_length", truncation=True, max_length=128)
    tokenized["labels"] = examples["class"]  # Rename class to labels
    return tokenized

# Apply tokenization
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set format
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

print("Tokenization completed.")
print("Train sample after tokenization:", train_dataset[0])
print("Train columns:", train_dataset.column_names)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/33600 [00:00<?, ? examples/s]

Map:   0%|          | 0/4200 [00:00<?, ? examples/s]

Map:   0%|          | 0/4200 [00:00<?, ? examples/s]

Tokenization completed.
Train sample after tokenization: {'input_ids': tensor([  101,  3021,  1996,  2417, 18278,  4149,  1037,  7937,  4744,  1998,
         2059,  2288,  1999,  1037,  2954,  2077, 12447,  2039,  2000, 17109,
         5745,  4165,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0, 

Cell 7: Load Model and Define Metrics

In [ ]:
# Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Define metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average="weighted")
    accuracy = accuracy_score(labels, predictions)
    return {"f1": f1, "accuracy": accuracy}

print("Model and metrics ready.")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model and metrics ready.


Cell 8: Set Training Arguments

In [ ]:
# Configure training
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    logging_dir="./logs",
    logging_steps=100,
    report_to="none",
    run_name="hate-speech-bert",
)

print("Training arguments set.")

Training arguments set.


Cell 9: Train Model

In [ ]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Train
trainer.train()

print("Training completed.")
from google.colab import drive
drive.mount('/content/drive')

# Define the path to save the model in your Google Drive
save_path = "/content/drive/MyDrive/fine_tuned_bert"

# Evaluate the model
test_results = trainer.evaluate(test_dataset)
print("Test Results:", test_results)

# Save model and tokenizer to Google Drive
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print("Model saved to Google Drive at:", save_path)
import os
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import numpy as np

# Create output directory
os.makedirs("plots", exist_ok=True)

# Extract logs
logs = trainer.state.log_history

# Separate loss and accuracy values
train_loss, eval_loss, eval_acc, steps = [], [], [], []

for log in logs:
    if 'loss' in log and 'epoch' in log:
        train_loss.append(log['loss'])
        steps.append(log['epoch'])
    if 'eval_loss' in log:
        eval_loss.append(log['eval_loss'])
    if 'eval_accuracy' in log:
        eval_acc.append(log['eval_accuracy'])

# Plot Loss
plt.figure(figsize=(10, 6))
plt.plot(steps, train_loss, label='Training Loss')
plt.plot(steps[:len(eval_loss)], eval_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training vs Validation Loss')
plt.legend()
plt.grid(True)
plt.savefig("plots/loss_curve.png")
plt.show()

# Plot Accuracy
plt.figure(figsize=(10, 6))
plt.plot(steps[:len(eval_acc)], eval_acc, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Validation Accuracy Over Epochs')
plt.legend()
plt.grid(True)
plt.savefig("plots/accuracy_curve.png")
plt.show()

# Get predictions
predictions = trainer.predict(test_dataset)
y_true = predictions.label_ids
y_pred = np.argmax(predictions.predictions, axis=1)

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
fig, ax = plt.subplots(figsize=(8, 6))
disp.plot(ax=ax)
plt.title("Confusion Matrix")
plt.savefig("plots/confusion_matrix.png")
plt.show()

# Classification Report
print("Classification Report:")
print(classification_report(y_true, y_pred))


NameError: name 'Trainer' is not defined

Cell 10: Evaluate and Save Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Define the path to save the model in your Google Drive
save_path = "/content/drive/MyDrive/fine_tuned_bert"

# Evaluate the model
test_results = trainer.evaluate(test_dataset)
print("Test Results:", test_results)

# Save model and tokenizer to Google Drive
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print("Model saved to Google Drive at:", save_path)


Test Results: {'eval_loss': 0.4229446351528168, 'eval_f1': 0.8836454199054805, 'eval_accuracy': 0.8840476190476191, 'eval_runtime': 30.617, 'eval_samples_per_second': 137.179, 'eval_steps_per_second': 8.59, 'epoch': 3.0}
Model saved.


Cell 11: Inference Example

In [ ]:
from transformers import pipeline

# Load the fine-tuned model
classifier = pipeline("text-classification", model="./fine_tuned_bert", tokenizer="./fine_tuned_bert")

# Run inference on user input
while True:
    sentence = input("Enter a sentence (or type '.exit' to quit): ")

    if sentence == ".exit":
        print("Exiting the program.")
        break

    result = classifier(sentence)
    print(f"Sentence: {sentence}\n   → {result}\n")


Device set to use cuda:0


Sentence: you are a shit person
   → [{'label': 'LABEL_1', 'score': 0.9006748199462891}]

Sentence: i hate you
   → [{'label': 'LABEL_0', 'score': 0.9935699701309204}]

Sentence: you will never win
   → [{'label': 'LABEL_2', 'score': 0.9607637524604797}]

Sentence: you were a bad person
   → [{'label': 'LABEL_2', 'score': 0.6830424070358276}]

Sentence: you are a shit
   → [{'label': 'LABEL_1', 'score': 0.9901621341705322}]

Sentence: kill everyone
   → [{'label': 'LABEL_0', 'score': 0.9952746629714966}]

Sentence: they are black and we should kill blaks
   → [{'label': 'LABEL_0', 'score': 0.9948168396949768}]

Sentence: who are u
   → [{'label': 'LABEL_2', 'score': 0.9963997602462769}]

Sentence: you are blind
   → [{'label': 'LABEL_2', 'score': 0.9874305725097656}]

Sentence: are you blind
   → [{'label': 'LABEL_2', 'score': 0.9812202453613281}]

Sentence: what have you done? are you blind?
   → [{'label': 'LABEL_1', 'score': 0.5671125650405884}]

Sentence: you hands are broken
   → 